<a href="https://www.kaggle.com/code/madeeltariq/api-gateway-rag?scriptVersionId=214630358" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# *Installing the required libraries*# 

In [ ]:
import subprocess
import sys
import os

def install_package(package_name):
    try:
        # Check if the package is installed
        subprocess.check_call([sys.executable, "-m", "pip", "show", package_name])
        print(f"{package_name} is already installed.")
    except subprocess.CalledProcessError:
        # If package is not installed, install it
        print(f"Installing {package_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package_name])
        print(f"{package_name} installation completed.")

def install_ngrok():
    # Check if ngrok is already downloaded and extracted
    if not os.path.exists('ngrok'):
        print("Downloading and installing ngrok...")
        subprocess.run(['wget', '-q', '-O', 'ngrok.zip', 'https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip'])
        subprocess.run(['unzip', '-o', 'ngrok.zip'])
        print("ngrok installed successfully.")
    else:
        print("ngrok is already installed.")

# Installing packages step-by-step
install_package("flask")
install_package("transformers")
install_package("pinecone-client")
install_package("python-dotenv")
install_package("flask-cors")
install_package("pyngrok")

# Install ngrok separately
install_ngrok()

print("Done With All installations >>>>>>>>>>>>>>>>> Move Forward")


# Imports in here

## **Code cell for Entering the ngrok API key For Ngrok implementation**

In [ ]:
import os
import getpass  # Import the getpass module to hide input text

# Function to create/update .env file with ngrok API key
def set_ngrok_api_key():
    # Ask user for the ngrok API key without displaying the input
    ngrok_api_key = getpass.getpass("Please enter your ngrok API key: ")
    
    # Check if .env file exists
    env_file = '.env'
    
    # If file exists, append the key; if not, create a new file
    if os.path.exists(env_file):
        with open(env_file, 'a') as file:
            file.write(f"NGROK_AUTHTOKEN={ngrok_api_key}\n")
    else:
        with open(env_file, 'w') as file:
            file.write(f"NGROK_AUTHTOKEN={ngrok_api_key}\n")
    
    print(f"ngrok API key added to {env_file}")

# Function to create/update .env file with Pinecone credentials
def set_pinecone_credentials():
    # Ask user for the Pinecone API key without displaying the input
    pinecone_api_key = getpass.getpass("Please enter your Pinecone API key: ")

    # Ask user for the Pinecone environment, with a default value
    pinecone_env = input("Please enter your Pinecone environment (e.g., us-west1-gcp) [default: us-east-1]: ") or "us-east-1"
    
    # Check if .env file exists
    env_file = '.env'
    
    # If file exists, append the keys; if not, create a new file
    if os.path.exists(env_file):
        with open(env_file, 'a') as file:
            file.write(f"PINECONE_API_KEY={pinecone_api_key}\n")
            file.write(f"PINECONE_ENV={pinecone_env}\n")
    else:
        with open(env_file, 'w') as file:
            file.write(f"PINECONE_API_KEY={pinecone_api_key}\n")
            file.write(f"PINECONE_ENV={pinecone_env}\n")
    
    print(f"Pinecone credentials added to {env_file}")

# Call the functions to set the API key and credentials
set_ngrok_api_key()
set_pinecone_credentials()


In [9]:
import os
from dotenv import load_dotenv  # Import the dotenv package to load .env file
from pinecone import Pinecone

# Function to get vector from Pinecone database based on ID
def get_vector_from_pinecone(file_id, index_name="vecotr", namespace_name="newCheck", pinecone_env="us-east-1"):
    # Load environment variables from .env file
    load_dotenv()  # This will load the variables from the .env file into the environment

    # Load Pinecone API key from the environment variables
    pinecone_api_key = os.getenv('PINECONE_API_KEY')
    if not pinecone_api_key:
        print("Check if ENV variables are set")

    # Ensure the API key is not None
    if not pinecone_api_key:
        # print("Error: Pinecone API key is missing from the environment variables.")
        return None

    # Initialize Pinecone client
    try:
        pc = Pinecone(api_key=pinecone_api_key)
        print(f"Pinecone client initialized with environment: {pinecone_env}")
    except Exception as e:
        print(f"Error initializing Pinecone client: {str(e)}")
        return None

    # Initialize the index
    try:
        index = pc.Index(index_name)
    except Exception as e:
        print(f"Error initializing Pinecone index: {str(e)}")
        return None

    # Query Pinecone to retrieve vector for the specific ID
    try:
        print(f"Retrieving vector for file ID: {file_id} from Pinecone...")

        # Querying Pinecone index for the specific ID using metadata filtering
        response = index.query(
            namespace=namespace_name,
            filter={"fileId": {"$eq": file_id}},
            id=file_id,
            top_k=1,
            include_values=True,
            include_metadata=True
        )

        # Check if result contains matches
        if response and response.get("matches"):
            # Display the first few vectors based on top_k
            for idx, vector_data in enumerate(response["matches"]):
                # print(f"Vector {idx + 1} ID: {vector_data['id']}")
                print(f"Pinecone Index: {index_name}")
                print(f"Pinecone Env: {pinecone_env}")
                # print(f"Metadata (fileId): {vector_data['metadata']['fileId']}")
                # print(f"Metadata (text): {vector_data['metadata']['text'][:100]}...")  # Showing first 100 chars of text
                print(f"Vector Values: {vector_data['values'][:5]}...")  # Showing first 5 values of the vector
                print(f"Size of Vector Values: {len(vector_data['values'])}")  # Printing the size of the vector values
            return response["matches"]

        else:
            print(f"No vector found for file ID: {file_id}")
            return None
    except Exception as e:
        print(f"Error retrieving vector for file ID {file_id}: {str(e)}")
        return None

# Example usage of the function
file_id = "676594520c3eb8c3272efa2c"  # Replace with the file ID you want to query

vector = get_vector_from_pinecone(file_id)



Pinecone client initialized with environment: us-east-1
Retrieving vector for file ID: 676594520c3eb8c3272efa2c from Pinecone...
Pinecone Index: vecotr
Pinecone Env: us-east-1
Vector Values: [0.029296875, -0.00984191895, -0.0322265625, -0.0369262695, 0.0473632812]...
Size of Vector Values: 1024


# To test this one is 

In [13]:
pip install pinecone-client requests python-dotenv


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [19]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec

# Load environment variables (for Pinecone API Key and other configurations)
load_dotenv()

# Initialize Pinecone client
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))

# Define the Pinecone index name and namespace
index_name = "vecotr"  # Ensure this matches your actual index name
namespace_name = "newCheck"  # Ensure this matches your namespace name

# Define the embedding model
model = "multilingual-e5-large"  # You can replace with any model of your choice

# Initialize Pinecone Index (Ensure your index exists or create it)
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,  # Use 1024 dimensions as per your embedding model
        metric='cosine',  # Use cosine similarity
        spec=ServerlessSpec(cloud='aws', region='us-west-2')
    )

# Get the index
index = pc.index(index_name)

# Function to generate embeddings using Pinecone's Inference API
def generate_embeddings(text):
    try:
        # Generate embeddings using Pinecone's inference API
        response = index.query(
            vector=[text],  # Input text to be embedded
            top_k=1,  # You can specify how many top matches you want (e.g., 1)
            namespace=namespace_name,
            include_values=True,  # Ensure you retrieve the vector values
            include_metadata=True,  # Include metadata if required
        )
        
        # Extract the vector values (assuming the first match)
        embeddings = response['matches'][0]['values']
        
        # Ensure the embeddings are of size 1024
        if len(embeddings) != 1024:
            raise ValueError(f"Generated embeddings have incorrect size: {len(embeddings)} (expected 1024)")

        return embeddings

    except Exception as e:
        print(f"Error generating embeddings: {e}")
        raise ValueError("Embedding generation failed.")

# Example: Using the function to get embeddings for a query
query = "What is the university name that is teaching the compiler construction course?"
query_vector = generate_embeddings(query)
print(f"Generated Vector: {query_vector[:5]}...")  # Display first 5 values of the vector


AttributeError: 'Pinecone' object has no attribute 'index'

In [15]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
import os
from pinecone import Pinecone

# Load the environment variables from .env file
load_dotenv()




# Function to retrieve and compare query vector with Pinecone vectors using cosine similarity
def find_similar_vectors(query):
    # Initialize the sentence transformer model that matches Pinecone's vector dimensionality (1024 in this case)
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # You can change this to a 1024-dimensional model
    
    # Vectorize the query
    query_vector = model.encode([query])[0]  # Convert the query into a vector

    # Fetch vector data from Pinecone for a sample file
    file_id = "676594520c3eb8c3272efa2c"  # Replace with your actual file ID
    result = get_vector_from_pinecone(file_id)

    if result is None:
        print("No results found.")
        return

    # Extract the first match from the result
    if isinstance(result, list) and len(result) > 0:
        match = result[0]  # Get the first match from the list of results

        # Extract the vector and metadata from the match
        stored_vector = match['values']
        metadata = match['metadata']

        # Check if the query vector and the stored vector have the same dimensionality
        if len(query_vector) != len(stored_vector):
            print(f"Error: Dimensionality mismatch! Query vector length: {len(query_vector)}, Stored vector length: {len(stored_vector)}")
            return

        # Calculate cosine similarity between the query vector and the Pinecone vector
        similarity = cosine_similarity([query_vector], [stored_vector])[0][0]

        # Print the similarity and the metadata of the most similar vector
        print(f"Cosine Similarity: {similarity:.4f}")
        print(f"Metadata (Text): {metadata.get('text', 'No text metadata available')}")
        print(f"Stored Vector: {stored_vector[:5]}...")  # Display first 5 values of the stored vector

        return similarity, metadata
    else:
        print("No valid matches found in the result.")
        return None

# Example usage
query = "What is the university name that is teaching the compiler construction course?"  # Example query
similarity, metadata = find_similar_vectors(query)


AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [2]:
from transformers import pipeline

def get_response_from_llm(query, model_name="EleutherAI/gpt-neo-1.3B"):
    """
    This function takes a user query, processes it using a free LLM from Hugging Face, 
    and returns the generated response.

    :param query: User input query.
    :param model_name: Model name to be used for text generation.
    :return: LLM's response to the query.
    """
    # Load the model pipeline
    generator = pipeline("text-generation", model=model_name, device=-1)  # Use CPU (-1) or GPU (set device to appropriate value)

    # Generate response
    response = generator(query, max_new_tokens=100, do_sample=True)
    
    # Return the generated response
    return response[0]["generated_text"]

# Example usage
user_query = "What is the significance of artificial intelligence?"
response = get_response_from_llm(user_query)
print("Response from the LLM:", response)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Response from the LLM: What is the significance of artificial intelligence?

Artificial intelligence is taking the place of humans in almost all fields and there is no reason to believe that artificial intelligence will replace us in the near future, if ever. Some people say that artificial intelligence will take over everything after we have achieved a peak and there is also a lot of hype around this topic. However, the truth is that we are not going to take over the world until we have built machines to mimic our intelligence. The way we work today makes it tough to be more


# Implemented Ngrok and CORS on top of the Flask API

In [ ]:
import os
import threading
import time
from flask import Flask, request, jsonify
from flask_cors import CORS  # Import CORS to handle cross-origin requests
from pyngrok import ngrok  # Import ngrok
from dotenv import load_dotenv  # Import the dotenv package to load .env file
from pinecone import Pinecone


# Load environment variables from .env file
load_dotenv()

# Function to get vector from Pinecone database based on ID
def get_vector_from_pinecone(file_id, index_name="vecotr", namespace_name="newCheck", pinecone_env="us-east-1"):
  
    
    # print("The File id given to the fucntion is :", file_id)
    # Load Pinecone API key from the environment variables
    pinecone_api_key = os.getenv('PINECONE_API_KEY')
    if not pinecone_api_key:
        print("Check if ENV variables are set")

    # Ensure the API key is not None
    if not pinecone_api_key:
        # print("Error: Pinecone API key is missing from the environment variables.")
        return None

    # Initialize Pinecone client
    try:
        pc = Pinecone(api_key=pinecone_api_key)
        # print(f"Pinecone client initialized with environment: {pinecone_env}")
    except Exception as e:
        print(f"Error initializing Pinecone client: {str(e)}")
        return None

    # Initialize the index
    try:
        index = pc.Index(index_name)
    except Exception as e:
        print(f"Error initializing Pinecone index: {str(e)}")
        return None

    # Query Pinecone to retrieve vector for the specific ID
    try:
        print(f"Retrieving vector for file ID: {file_id} from Pinecone...")

        # Querying Pinecone index for the specific ID using metadata filtering
        response = index.query(
            namespace=namespace_name,
            filter={"fileId": {"$eq": file_id}},
            id=file_id,
            top_k=1,
            include_values=True,
            include_metadata=True
        )

        # Check if result contains matches
        if response:
            return response

        else:
            print(f"No vector found for file ID: {file_id}")
            return None
    except Exception as e:
        print(f"Error retrieving vector for file ID {file_id}: {str(e)}")
        return None



# Initialize Flask app
app = Flask(__name__)

# Enable CORS for all routes
CORS(app)

# Health check endpoint with request body and logging
@app.route('/get_model_response', methods=['POST'])
def get_model_response():
    print("[DEBUG] Received request at /get_model_response")
    try:
        # Log the received request data
        data = request.get_json()
        print(f"[DEBUG] Request body: {data}")

        # Validate the request body
        if not data:
            print("[ERROR] No JSON data received in the request.")
            return jsonify({"error": "Request body must be JSON"}), 400

        file_id = data.get("file_id")
        if not file_id:
            print("[ERROR] 'file_id' is missing in the request body.")
            return jsonify({"error": "'file_id' is required"}), 400

        # Call the function to get vector from Pinecone
        vector_response = get_vector_from_pinecone(file_id)
        if vector_response and vector_response.get("matches"):
            matches = vector_response["matches"]
            if matches:
                # Display the first few vectors based on top_k
                for idx, vector_data in enumerate(matches):
                    print(f"Vector Values: {vector_data['values'][:5]}...")
                    print(f"Size of Vector Values: {len(vector_data['values'])}")
                
                # Only send back the vector_data part
                # print("the vector data is:", vector_data['values'])
                return jsonify({"status": "success", "data": vector_data['values']}), 200

        else:
            print("[WARNING] No vector found for the provided file_id.")
            return jsonify({"status": "error", "message": "No vector found for the given file_id"}), 404

    except Exception as e:
        print(f"[ERROR] Exception occurred: {str(e)}")
        return jsonify({"status": "error", "message": "An error occurred"}), 500


@app.route('/test', methods=['GET'])
def test():
    return jsonify({"message": "Flask server is running!"})

# Function to run Flask server
def run_flask():
    # print("Flask server is listening on port 5000...")
    app.run(host="0.0.0.0", port=5000)  # Running on port 5000

# Function to start ngrok and expose the Flask server
def run_ngrok():
    # Authenticate ngrok using the API key from the .env file
    ngrok_auth_token = os.getenv("NGROK_AUTHTOKEN")
    if ngrok_auth_token:
        ngrok.set_auth_token(ngrok_auth_token)
        # print("Ngrok authenticated successfully.")
    else:
        print("Ngrok authentication failed. Please check your API key.")
        return None

    # Open a ngrok tunnel to the Flask app on port 5000
    public_url = ngrok.connect(5000)
    print(f"Ngrok public URL: {public_url}")
    return public_url

# Start Flask server in a separate thread to keep the notebook running
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True  # Allow thread to terminate when the program exits
flask_thread.start()

# Start ngrok and print the URL where Flask is accessible
public_url = run_ngrok()

# Keep the notebook running to maintain the server
while True:
    time.sleep(1)  # Just keeps the notebook running


# *Working Prototype for flask listening***

In [ ]:
import os
from flask import Flask, request, jsonify
import threading
import time

# Initialize Flask app
app = Flask(__name__)

# Health check endpoint
@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({"status": "Flask server is running Greate!"})

@app.route('/test', methods=['GET'])
def test():
    return jsonify({"message": "Flask server is running!"})

# Function to run Flask server
def run_flask():
    print("Flask server is listening on port 8000...")
    app.run(host="0.0.0.0", port=9000)  # Running on port 8000

# Start Flask server in a separate thread to keep the notebook running
flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True  # Allow thread to terminate when the program exits
flask_thread.start()

# Print out the URL where Flask is running
print("Flask server should be accessible at: http://0.0.0.0:8000")

# Keep the notebook running to maintain the server
while True:
    time.sleep(1)  # Just keeps the notebook running


# How to send request from the JavaScript Server Side 